In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import contextily as ctx
from geopy.distance import geodesic
from datetime import datetime
import numpy as np

In [2]:
filename_DD = '../data/nextbike/trips_Dresden 2025-01-26_19-59-31.pkl'
filename_FB = '../data/nextbike/trips_Freiburg 2025-01-26_19-59-31.pkl'
df_DD = pd.read_pickle(filename_DD)
df_FB = pd.read_pickle(filename_FB)

In [3]:
original_cols = df_DD.columns

In [4]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
pd.set_option('display.max_colwidth', 400)

# DD

In [ ]:
df_DD.head()

In [6]:
tmp = df_DD[[ "name_return", 'bike_return']].value_counts(dropna=False)

In [ ]:
tmp.head(40)

In [8]:
grouping = df_DD[(df_DD.uid_rent == df_DD.uid_return)].groupby(["datetime_hour_return", "duration_min_bin"]).size().sort_values(ascending=False)

In [ ]:
print(grouping)

In [10]:
outlier_days_DD = ["2024-02-12", "2024-02-13", "2024-02-14", "2024-03-27", "2024-09-19", "2024-09-20", "2024-09-22", "2024-09-24", "2024-10-09", "2024-10-27"]
outlier_days_DD = [pd.to_datetime(x).date() for x in outlier_days_DD]

In [ ]:
flt_DD = (df_DD.date_return.isin(outlier_days_DD)) & (df_DD.date_rent.isin(outlier_days_DD)) & (df_DD.uid_rent == df_DD.uid_return)  & (df_DD.duration_min<=60 )
df_DD[flt_DD].groupby("duration_min_bin").size().sort_values(ascending=False)

In [ ]:
len(df_DD[flt_DD])/len(df_DD)

In [13]:
df_DD = df_DD[~flt_DD]

In [14]:
grouping = df_DD[(df_DD.uid_rent == df_DD.uid_return)].groupby(["datetime_hour_return", "duration_min_bin"]).size().sort_values(ascending=False)

In [ ]:
flt_DD_2 = (df_DD.duration_min<=5) & (df_DD.uid_rent == df_DD.uid_return)
len(df_DD[flt_DD_2])/len(df_DD)

In [ ]:
0.004175813250096153 + 0.011861378594605957

In [17]:
df_DD = df_DD[~flt_DD_2]

In [18]:
# grouping

In [19]:
# df_DD.groupby(['datetime_return', 'uid_return', 'name_return']).size().sort_values(ascending=False).head(40)

# FB

In [ ]:
df_FB.head()

In [21]:
# already accounted for by 24 hour filter 
# missing_FB_adays = ["2023-06-26", "2023-06-27", "2023-06-29", "2023-06-30"]
# missing_FB_days = [pd.to_datetime(x).date() for x in missing_FB_days]

In [22]:
# missing_FB_days

In [23]:
#tmp = df_FB.groupby("date_return").size().sort_index()

In [24]:
grouping = df_FB[(df_FB.uid_rent == df_FB.uid_return) & (df_FB.duration_min<=60)].groupby(["datetime_hour_return", "duration_min_bin"]).size().sort_values(ascending=False)

In [ ]:
grouping[grouping > 10].head(30).sort_values(ascending=False)

In [26]:
outlier_days_FB = ["2023-06-04", "2023-06-08",  "2023-06-24","2024-09-20"]
outlier_days_FB = [pd.to_datetime(x).date() for x in outlier_days_DD]

In [ ]:
flt_FB = (df_FB.date_return.isin(outlier_days_FB)) & (df_FB.date_rent.isin(outlier_days_FB)) & (df_FB.uid_rent == df_FB.uid_return)  & (df_FB.duration_min<=60 )
df_FB[flt_FB].groupby("duration_min_bin").size().sort_values(ascending=False)

In [ ]:
len(df_FB[flt_FB])/len(df_FB)*100

In [29]:
df_FB = df_FB[~flt_FB]

In [30]:
grouping = df_FB[(df_FB.uid_rent == df_FB.uid_return) & (df_FB.duration_min<=60)].groupby(["datetime_hour_return", "duration_min_bin"]).size().sort_values(ascending=False)

In [ ]:
grouping[grouping > 10].head(30).sort_values(ascending=False)

In [ ]:
flt_FB_2 = (df_FB.duration_min<=5) & (df_FB.uid_rent == df_FB.uid_return)
len(df_FB[flt_FB_2])/len(df_FB)

In [ ]:
0.034818869529613335 + 0.002905363626274317

In [34]:
df_FB = df_FB[~flt_FB_2]

# Too fast and mass returns

In [ ]:
df_DD.head()
df_DD['speed_kmh'] = df_DD.distance/1000/df_DD.duration_min*60
#df_DD.speed_kmh.plot(kind='hist', bins=100, xlim=(0, 1000))
df_DD[df_DD.speed_kmh>50].groupby("name_return", dropna=False).size().sort_values(ascending=False)
len(df_DD[df_DD.speed_kmh>50])/len(df_DD)
#df_DD[df_DD.speed_kmh>50]

In [ ]:
grouping = df_DD[df_DD.name_return =="MOBIpunkt TU Dresden"].groupby(["datetime_return", "name_return"]).size().sort_values(ascending=False)
# df_DD[df_DD.datetime_return == pd.to_datetime("2024-03-27 15:02:00")] # tracking disturbance, but these are real trips, there was tracking with a delay
df_DD.groupby("name_return").size().sort_values(ascending=False).head(10)
df_DD.name_return.unique()
grouping = df_DD[~df_DD.name_return.astype(str).str.contains("BIKE")].groupby(["datetime_return", "name_return"]).size().sort_values(ascending=False)
grouping

In [ ]:
grouping[grouping.index.get_level_values("name_return").str.contains("Haupt")]

In [ ]:
df_hbf = df_DD[df_DD.name_return.astype(str).str.contains("Haupt")].copy()

In [ ]:
df_hbf_counts = df_hbf.datetime_hour_return.value_counts().sort_index()

In [42]:
df_hbf_counts = df_hbf_counts[df_hbf_counts>10]

In [ ]:


grouping = grouping[grouping>10]
# filter out the bike stations of df_DD based on the multi index
flt_DD = df_DD.set_index(["datetime_return", "name_return"]).index.isin(grouping.index)
df_DD.loc[flt_DD, "is_mass_return"]=True
df_DD.is_mass_return.fillna(False, inplace=True)
df_DD.loc[df_DD.speed_kmh>50, "is_too_fast"]=True
df_DD.is_too_fast.fillna(False, inplace=True)
df_DD[['is_too_fast', 'is_mass_return']].value_counts()
df_DD.groupby("is_mass_return")['speed_kmh'].mean()
# correlation of 2 boolean columns
df_DD[['is_too_fast', 'is_mass_return']].corr()
df_DD[df_DD.is_mass_return==True].groupby("name_return").size().sort_values(ascending=False)
df_DD['return_hour_int'] = df_DD.datetime_return.dt.hour
df_DD[df_DD.is_mass_return==True].groupby("return_hour_int").size().sort_index()
df_DD.is_mass_return.value_counts(normalize=True)

In [ ]:
df_FB.head()
df_FB['speed_kmh'] = df_FB.distance/1000/df_FB.duration_min*60
df_FB.speed_kmh.plot(kind='hist', bins=100, xlim=(0, 1000))
df_FB[df_FB.speed_kmh>50].groupby("name_return", dropna=False).size().sort_values(ascending=False)
len(df_FB[df_FB.speed_kmh>50])/len(df_FB)
df_FB[df_FB.speed_kmh>50]
grouping = df_FB[df_FB.name_return =="MOBIpunkt TU Dresden"].groupby(["datetime_return", "name_return"]).size().sort_values(ascending=False)
# df_FB[df_FB.datetime_return == pd.to_datetime("2024-03-27 15:02:00")] # tracking disturbance, but these are real trips, there was tracking with a delay
df_FB.groupby("name_return").size().sort_values(ascending=False).head(10)
df_FB.name_return.unique()
grouping = df_FB[~df_FB.name_return.astype(str).str.contains("BIKE")].groupby(["datetime_return", "name_return"]).size().sort_values(ascending=False)
grouping
grouping = grouping[grouping>10]
# filter out the bike stations of df_FB based on the multi index
flt_FB = df_FB.set_index(["datetime_return", "name_return"]).index.isin(grouping.index)
df_FB.loc[flt_FB, "is_mass_return"]=True
df_FB.is_mass_return.fillna(False, inplace=True)
df_FB.loc[df_FB.speed_kmh>50, "is_too_fast"]=True
df_FB.is_too_fast.fillna(False, inplace=True)
df_FB[['is_too_fast', 'is_mass_return']].value_counts()
df_FB.groupby("is_mass_return")['speed_kmh'].mean()
# correlation of 2 boolean columns
df_FB[['is_too_fast', 'is_mass_return']].corr()

df_FB['return_hour_int'] = df_FB.datetime_return.dt.hour

df_FB.is_mass_return.value_counts(normalize=True)


In [ ]:
df_FB[df_FB.is_mass_return==True].groupby("name_return").size().sort_values(ascending=False)

In [ ]:
df_FB[df_FB.is_mass_return==True].groupby("return_hour_int").size().sort_index()

In [ ]:
df_FB.is_mass_return.value_counts(normalize=True)

In [ ]:
len(df_DD[df_DD.is_too_fast == True])/len(df_DD)

In [ ]:
len(df_FB[df_FB.is_too_fast == True])/len(df_FB)

In [262]:
df_DD = df_DD[df_DD.is_too_fast==False]
df_FB = df_FB[df_FB.is_too_fast==False]
df_DD = df_DD[df_DD.is_mass_return==False]
df_FB = df_FB[df_FB.is_mass_return==False]


In [ ]:
len(df_DD)

In [ ]:
len(df_FB)

In [266]:
df_DD = df_DD[original_cols]
df_FB = df_FB[original_cols]

In [267]:
time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
df_DD.to_pickle(f'../data/nextbike/trips_Dresden {time}.pkl')
df_FB.to_pickle(f'../data/nextbike/trips_Freiburg {time}.pkl')

In [ ]:
time